# Manuels Forms Creator

This notebook uses OAuth to access Google Forms API. While the client_secret is part of this notebook, the client configuration needs auhtenticating google user accounts to be allow-listed before use.

## Authenticate

Allow this notebook to access the google account of the user.

In [ ]:
from datetime import datetime
from google.colab import auth
auth.authenticate_user()

For google sheets we can use the colab credentials.

In [ ]:
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

For the forms API we need to use additional credentials via an OAuth flow.

In [ ]:
from googleapiclient.discovery import build

In [ ]:
%%writefile client_secret.json
{"installed":{"client_id":"..."}}

In [ ]:
import google.auth.transport.requests
from google_auth_oauthlib.flow import Flow

flow = Flow.from_client_secrets_file(
    'client_secret.json',
    scopes=[
        'https://www.googleapis.com/auth/forms.body',
        'https://www.googleapis.com/auth/forms.body.readonly',
        'https://www.googleapis.com/auth/drive.metadata.readonly'
    ],
    redirect_uri='urn:ietf:wg:oauth:2.0:oob'
)

The next cell performs the OAuth login. It generates a special authorization URL for the user to follow. The user has to grant the application access and receives an authorization code. This code has to be pasted here, into the input field. With this authorization code the application can acquire the required credentials to access the forms API on the users behalf.

If the OAuth delegation does not work, please contact Manuel. Since the OAuth client is an unofficial client that has not gone through google review, it requires all users to be allow-listed in advance.

In [ ]:
auth_url, _ = flow.authorization_url(prompt='consent')
print("Go to this URL:\n", auth_url)

# Paste the code you get after authorizing:
code = input('Enter the authorization code: ')
flow.fetch_token(code=code)

creds = flow.credentials
print(creds.granted_scopes)

Now that we have the OAuth credentials, we build new clients for forms and drive API.

In [ ]:
drive_service = build('drive', 'v3', credentials=creds)
forms_service = build('forms', 'v1', credentials=creds)

## List all Forms

Lets see what forms we have.

In [ ]:
query = "mimeType='application/vnd.google-apps.form'"

results = drive_service.files().list(q=query, fields="files(id, name)").execute()
forms = results.get('files', [])

if not forms:
    print("No forms found.")
else:
    print("Forms:")
    for form in forms:
        print(f"Name: {form['name']}, ID: {form['id']}")


## Get Form details

If we know the `form_id` we can load it and view its details.

In [ ]:
form_id = '1hiJQ_4eWMeY3jup57p1rVZ8Na0XU8I8nCV7sCTrjShB' #@param {type: "string"}

try:
  form_details = forms_service.forms().get(formId=form_id).execute()
  print(form_details)
  print("Form Title:", form_details.get('info', {}).get('title'))
  print("Form ID:", form_details.get('formId'))
  print("Items:")
  for item in form_details.get('items', []):
    print(f"- {item.get('title', 'Untitled')} (ID: {item.get('itemId')})")

except Exception as err:
  print(f"Error: {err}")

## Create new Form

We generate a new form via the API. In the future we might reuse an existing form.

In [ ]:
now = datetime.now()
form_metadata = {
    "info": {
        "title": f"Sample Form from Colab {now:%Y%m%d%H%M%S}",
        "documentTitle": f"Colab Google Form {now:%Y%m%d%H%M%S}"
    }
}

new_form = forms_service.forms().create(body=form_metadata).execute()

form_id = new_form['formId']
print("Form created:", new_form['responderUri'], "as", form_id)

## Add options to Form

We add a new question to the form. The question has a few options. Each option can only have a single image. To display more images in the future, we might need to collage multiple images together as one. But then we cannot use the `sourceUri` property anymore.

In [ ]:
form_details = forms_service.forms().get(formId=form_id).execute()
form_items = form_details.get('items', [])

question_item = {
    "createItem": {
        "item": {
            "title": "Who is your favorite celebrity?",
            "questionItem": {
                "question": {
                    "required": True,
                    "choiceQuestion": {
                        "type": "RADIO",
                        "options": [
                            {
                                "value": "Angourie Rice",
                                "image": {
                                    "sourceUri": "https://tse1.mm.bing.net/th?id=OIP.nesaW00tFv6kIWc9NuLMOAHaLH&pid=Api&P=0&w=300&h=300"
                                }
                            },
                            {
                                "value": "Emma Watson",
                                "image": {
                                    "sourceUri": "https://tse4.mm.bing.net/th?id=OIP.HP9zdd9h6pobkEXAHhcY-gHaJZ&pid=Api&P=0&w=300&h=300"
                                }
                            },
                            {
                                "value": "Jenna Ortega",
                                "image": {
                                    "sourceUri": "https://tse1.mm.bing.net/th?id=OIP.dde1du-cxWb3Whspz5ibdQHaLG&pid=Api&P=0&w=300&h=300"
                                }
                            },
                            {
                                "value": "Millie Bobby Brown",
                                "image": {
                                    "sourceUri": "https://tse2.mm.bing.net/th?id=OIP.QxAx1wmOuvGGRgwy2M7dmgHaJ4&pid=Api&P=0&w=300&h=300"
                                }
                            },
                        ],
                    }
                }
            }
        },
        "location": {
            "index": len(form_items)
        }
    }
}

request = {"requests": [question_item]}
result = forms_service.forms().batchUpdate(formId=form_id, body=request).execute()
print(f"Added question: {result}")
